In [1]:
'''poker engine'''

# pylint: disable=E1101, E1601, W0612

import copy
import datetime
import funcs_decision_point as fdp
import funcs_other as fother
import funcs_range as frange
import funcs_sql_del as fsqld
import funcs_sql_ins as fsqli
import funcs_sql_sel as fsqls
import funcs_sql_upd as fsqlu
import gc
import json
import mysql.connector
import numpy as np
import os
import pickle
import pypokerengine as p
from pypokerengine.api.emulator import Emulator
from pypokerengine.engine.game_evaluator import GameEvaluator as ge
from pypokerengine.engine.round_manager import MessageBuilder as msgb
from pypokerengine.utils.game_state_utils import DataEncoder as de

In [2]:
gc.collect()

22

In [3]:
json_data = open(file='./settings.json', mode='r')
settings = json.load(json_data)
json_data.close()

settings

{'ante_amount': 0,
 'deuces_path': 'c:/ProgramData/Anaconda3/Lib/site-packages/deuces/',
 'game_state_path': '../../game_state/',
 'holdem_calc_path': '../holdem_calc/',
 'max_round': 50,
 'nr': 0,
 'player_num': 6,
 'pot': 0,
 'side': 0,
 'small_blind_amount': 10,
 'stack': 2000,
 'step': 0,
 'sql_checksum': '../../sql/checksum/',
 'sql_database': 'poker_reinforcement_model',
 'sql_host': '127.0.0.1',
 'sql_path': '../../sql/reinforcement_model/',
 'sql_user': 'root'}

In [4]:
seed = np.random.randint(low=1, high=100000)
np.random.seed(seed=seed)
seed

seed = 18230
np.random.seed(seed=seed)
seed

18230

In [5]:
poker_db = mysql.connector.connect(user=settings['sql_user'],
                                   host=settings['sql_host'],
                                   database=settings['sql_database'])
database = settings['sql_database']
sql_path = settings['sql_path']
sql_checksum = settings['sql_checksum']
game_state_path = settings['game_state_path']

In [6]:
# query available tables from database
tables = fsqls.sql_select_tables(poker_db=poker_db,
                                 database=database,
                                 sql_path=sql_path)
# delete content of available tables in database
for table in tables:
    fsqld.sql_delete_allrows(poker_db=poker_db,
                             database=database,
                             table=table,
                             sql_path=sql_path)

print('Done')

Done


In [7]:
emulator = Emulator()

In [8]:
# basic settings of the game
ante_amount = settings['ante_amount']
max_round = settings['max_round']
player_num = settings['player_num']
small_blind_amount = settings['small_blind_amount']

players_info = {
    'uuid-0': {'name': 'player0', 'stack': settings['stack']},
    'uuid-1': {'name': 'player1', 'stack': settings['stack']},
    'uuid-2': {'name': 'player2', 'stack': settings['stack']},
    'uuid-3': {'name': 'player3', 'stack': settings['stack']},
    'uuid-4': {'name': 'player4', 'stack': settings['stack']},
    'uuid-5': {'name': 'player5', 'stack': settings['stack']}
}

In [9]:
# set basic rules of the game
emulator.set_game_rule(player_num=player_num,
                       max_round=max_round,
                       small_blind_amount=small_blind_amount,
                       ante_amount=ante_amount)

# initialize game
game_state = emulator.generate_initial_game_state(players_info)

In [10]:
# start building new dataset
start_time = datetime.datetime.now()
events = [{'type': ''}]

# if process start at some exported state then use the game_state['round_count'] value to identify where to
# start from. in that case the inital game state round_count should be shifted with the value of the loaded
# game state round_count. IMPORTANT: the game should be deleted from database (every related data which refers
# to the given game) where the game id is the loaded game state round_count
# game_state = pickle.load(file=open(file=game_state_path + 'start_of_game_state_' + str(game_state['round_count']) + '.obj',
#                                    mode='rb'))
while events[-1]['type'] != 'event_game_finish':    
    game_state, events = emulator.start_new_round(game_state)
    
    #### delete and dump game_state
    for file in os.listdir(path=game_state_path):
        os.remove(path=game_state_path + file)
    
    pickle.dump(obj=game_state, file=open(file=game_state_path + 'start_of_game_state_' + str(game_state['round_count']) + '.obj',
                                          mode='wb'))
    
    # log round count
    if game_state['round_count'] == 1 or game_state['round_count'] % 10 == 0:
        print('Running', game_state['round_count'], 'game')
    
    # load basic table settings into database
    for player in game_state['table'].seats.players:
        #### calculate stack for players, SB and BB amounts are modified due to the structure of available data
        if game_state['table'].sb_pos() == int(player.uuid.split('-')[1]):
            stack = player.stack + game_state['small_blind_amount']
        elif game_state['table'].bb_pos() == int(player.uuid.split('-')[1]):
            stack = player.stack + 2 * game_state['small_blind_amount']
        else:
            stack = player.stack

        position = (6 + (int(player.uuid.split('-')[1]) - game_state['table'].dealer_btn)) % 6 # calculate position

        #### insert available data into table containing the descriptions of the games
        fsqli.sql_insert_games(poker_db=poker_db,
                               database=database,
                               sql_path=sql_path,
                               index=game_state['round_count'],
                               player_num=game_state['table'].seats.count_active_players(),
                               small_blind_amount=game_state['small_blind_amount'],
                               ante_amount=ante_amount,
                               uuid=player.uuid,
                               name=player.name,
                               stack=stack,
                               position=position,
                               card1='',
                               card2='',
                               hand_db_format='',
                               flop1='',
                               flop2='',
                               flop3='',
                               turn='',
                               river='',
                               final_stack=0)

    for player in game_state['table'].seats.players:
        #### calculate hole cards for the players
        card_list = [
            player.hole_card[0].SUIT_MAP[player.hole_card[0].suit] +
            player.hole_card[0].RANK_MAP[player.hole_card[0].rank],
            player.hole_card[1].SUIT_MAP[player.hole_card[1].suit] +
            player.hole_card[1].RANK_MAP[player.hole_card[1].rank]
        ]
        card_list.sort()
        card1 = card_list[0]
        card2 = card_list[1]
        if card1[0] == card2[0]:
            hand_db_format = card1[-1] + card2[-1] + 's'
        else:
            hand_db_format = card1[-1] + card2[-1] + 'o'

        #### update table containing the descriptions of games with hole cards
        fsqlu.sql_update_games_cards(poker_db=poker_db,
                                     database=database,
                                     sql_path=sql_path,
                                     index=game_state['round_count'],
                                     uuid=player.uuid,
                                     card1=card1,
                                     card2=card2,
                                     hand_db_format=hand_db_format)
    
    # load smallblind and bigblind data into database
    nr = settings['nr']
    pot = settings['pot']
    step = settings['step']

    for player in game_state['table'].seats.players:
        if player.action_histories != [] \
        and player.action_histories[0]['action'] in ['SMALLBLIND', 'BIGBLIND']:
            phase = de.encode_street(street=game_state['street'])['street'] # calculate phase
            position = (6 + (int(player.uuid.split('-')[1]) - game_state['table'].dealer_btn)) % 6 # calculate position

            #### calculate stack for players, SB and BB amounts are modified due to the structure of available data
            if player.action_histories[0]['action'] == 'SMALLBLIND':
                stack = player.stack + game_state['small_blind_amount']
            elif player.action_histories[0]['action'] == 'BIGBLIND':
                stack = player.stack + 2 * game_state['small_blind_amount']
            else:
                stack = player.stack

            #### insert available data into table containing the game histories
            fsqli.sql_insert_history(poker_db=poker_db,
                                     database=database,
                                     sql_path=sql_path,
                                     phase=phase,
                                     nr=nr,
                                     step=step,
                                     uuid=player.uuid,
                                     position=position,
                                     stack=stack,
                                     stack_range=frange.range_stack(stack=stack,
                                                                    small_blind_amount=game_state['small_blind_amount']),
                                     pot=pot,
                                     pot_range=frange.range_pot(pot=pot,
                                                                small_blind_amount=game_state['small_blind_amount']),
                                     flop1='',
                                     flop2='',
                                     flop3='',
                                     turn='',
                                     river='',
                                     action=player.action_histories[0]['action'],
                                     amount=player.action_histories[0]['amount'],
                                     new_stack=player.stack,
                                     new_stack_range=frange.range_stack(stack=player.stack,
                                                                        small_blind_amount=game_state['small_blind_amount']),
                                     new_pot=pot + player.action_histories[0]['amount'],
                                     new_pot_range=frange.range_pot(pot=pot + player.action_histories[0]['amount'],
                                                                    small_blind_amount=game_state['small_blind_amount']),
                                     amount_potrate=frange.range_potrate(amount=player.action_histories[0]['amount'],
                                                                         pot=pot,
                                                                         action=player.action_histories[0]['action']))

            nr = nr + 1
            step = step + 1
            pot = pot + player.action_histories[0]['amount']

    # game phase
    while events[-1]['type'] != 'event_round_finish' \
    and events[-1]['type'] != 'event_game_finish':
        if game_state['next_player'] != 'not_found' and game_state['street'] not in [4, 5]:
            #### calculate game parameters for current state and community cards
            next_player = game_state['next_player']
            msg = msgb.build_ask_message(player_pos=next_player,
                                         state=game_state)['message']
            community_cards = fother.community_cards_eval(board=msg['round_state']['community_card'])
            position = (6 + (next_player - game_state['table'].dealer_btn)) % 6 # calculate position
            phase = de.encode_street(street=game_state['street'])['street'] # calculate phase
            
            #### calculate stack, hole cards and win rate for current player and state
            stack = game_state['table'].seats.players[next_player].stack
            hole_cards = msg['hole_card']
            win_rate = fother.winrate_calc(hole_cards=hole_cards,
                                           community_cards=community_cards,
                                           nb_player=game_state['table'].seats.count_active_players())

            #### calculate pot (main + side)
            pot = settings['pot']
            for item in ge.create_pot(game_state['table'].seats.players):
                pot = pot + item['amount']
            
            #### calculate action for current decision point
            action, bet_amount = \
                    fdp.decision_point_based_action(poker_db=poker_db,
                                                    database=database,
                                                    sql_path=sql_path,
                                                    phase=phase,
                                                    nr=nr,
                                                    step=step,
                                                    position=position,
                                                    stack=stack,
                                                    pot=pot,
                                                    flop1=community_cards[0],
                                                    flop2=community_cards[1],
                                                    flop3=community_cards[2],
                                                    turn=community_cards[3],
                                                    river=community_cards[4],
                                                    valid_actions=msg['valid_actions'],
                                                    win_rate=win_rate,
                                                    small_blind_amount=game_state['small_blind_amount'],
                                                    seed=seed)

            #### apply calculated action for current decision point
            game_state, events = emulator.apply_action(game_state=game_state,
                                                       action=action,
                                                       bet_amount=bet_amount)

            #### calculate new parameters
            if game_state['next_player'] != 'not_found' and game_state['street'] not in [4, 5]:
                new_stack = game_state['table'].seats.players[next_player].stack
                amount = stack - new_stack
                new_pot = pot + amount
            else:
                for event in events:
                    if event['type'] == 'event_round_finish':
                        new_pot = event['round_state']['pot']['main']['amount']
                        if event['round_state']['pot']['side'] != []:
                            for item in event['round_state']['pot']['side']:
                                new_pot = new_pot + item['amount']
                
                amount = new_pot - pot
                new_stack = stack -amount
            
            new_phase = de.encode_street(street=game_state['street'])['street'] # calculate new phase

            #### insert available data into table containing the game histories
            fsqli.sql_insert_history(poker_db=poker_db,
                                     database=database,
                                     sql_path=sql_path,
                                     phase=phase,
                                     nr=nr,
                                     step=step,
                                     uuid='uuid-' + str(next_player),
                                     position=position,
                                     stack=stack,
                                     stack_range=frange.range_stack(stack=stack,
                                                                    small_blind_amount=game_state['small_blind_amount']),
                                     pot=pot,
                                     pot_range=frange.range_pot(pot=pot,
                                                                small_blind_amount=game_state['small_blind_amount']),
                                     flop1=community_cards[0],
                                     flop2=community_cards[1],
                                     flop3=community_cards[2],
                                     turn=community_cards[3],
                                     river=community_cards[4],
                                     action=action.upper(),
                                     amount=amount,
                                     new_stack=new_stack,
                                     new_stack_range=frange.range_stack(stack=new_stack,
                                                                        small_blind_amount=game_state['small_blind_amount']),
                                     new_pot=new_pot,
                                     new_pot_range=frange.range_pot(pot=new_pot,
                                                                    small_blind_amount=game_state['small_blind_amount']),
                                     amount_potrate=frange.range_potrate(amount=amount,
                                                                         pot=pot,
                                                                         action=action.upper()))

            if new_phase == phase:
                nr = nr + 1
            else:
                nr = 0

            step = step + 1
    
    # summarize and finalize round
    for event in events:
        if event['type'] == 'event_round_finish':
            #### update final stack and community cards in table of games
            for seat in event['round_state']['seats']:
                fsqlu.sql_update_games_final_stack(poker_db=poker_db,
                                                   database=database,
                                                   sql_path=sql_path,
                                                   index=game_state['round_count'],
                                                   uuid=seat['uuid'],
                                                   final_stack=seat['stack'])
            
            community_cards = fother.community_cards_eval(board=event['round_state']['community_card'])
            fsqlu.sql_update_games_board(poker_db=poker_db,
                                         database=database,
                                         sql_path=sql_path,
                                         index=game_state['round_count'],
                                         flop1=community_cards[0],
                                         flop2=community_cards[1],
                                         flop3=community_cards[2],
                                         turn=community_cards[3],
                                         river=community_cards[4])

            #### update possible moves features in possible moves
            fsqlu.sql_update_possible_moves(poker_db=poker_db,
                                            database=database,
                                            sql_path=sql_path,
                                            index=game_state['round_count'])

    # rebuy if necesseray
    for player in game_state['table'].seats.players:
        if player.stack <= 2 * small_blind_amount:
            player.append_chip(2000 * np.random.randint(low=1, high=11))
    
    # run checksum function to check the generated database
    if game_state['round_count'] == 1:
        checksum = 0
    else:
        checksum_dict, checksum = fother.run_checksum(poker_db=poker_db,
                                                      database=database,
                                                      sql_checksum=sql_checksum)
    
    if checksum != 0:
        print('\nSum of checksums should be 0. Something went wrong')
        print('Round:', game_state['round_count'], 'Checksum:', checksum, '\n', checksum_dict)
        break
    
    # elapsed time
    if game_state['round_count'] == 1 or game_state['round_count'] % 10 == 0:
        print('- Elapsed time:', datetime.datetime.now() - start_time,
              'Avg time for a round:', (datetime.datetime.now() - start_time) / game_state['round_count'])

print('Done')

Running 1 game
- Elapsed time: 0:00:07.316312 Avg time for a round: 0:00:07.316312
Running 10 game
- Elapsed time: 0:01:12.661971 Avg time for a round: 0:00:07.266197
Running 20 game
- Elapsed time: 0:02:56.231106 Avg time for a round: 0:00:08.811555
Running 30 game
- Elapsed time: 0:08:07.067253 Avg time for a round: 0:00:16.235575
Running 40 game
- Elapsed time: 0:12:14.225071 Avg time for a round: 0:00:18.355627
Running 50 game
- Elapsed time: 0:24:11.580729 Avg time for a round: 0:00:29.031615
Done


In [11]:
checksum_dict, checksum = fother.run_checksum(poker_db=poker_db,
                                              database=database,
                                              sql_checksum=sql_checksum)

print(checksum, '\n', checksum_dict)

0.0 
 {'checksum_amount_pot': 0.0, 'checksum_ante': 0.0, 'checksum_player_num': 0.0, 'checksum_position': 0.0, 'checksum_pot': 0.0, 'checksum_sb': 0.0, 'checksum_stack': 0.0, 'checksum_stack_final_stack': 0.0, 'checksum_stack_new_stack': 0.0, 'checksum_stack_vs_stack': 0.0, 'checksum_uuid_name': 0.0}
